In [1]:
import pandas as pd 
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams
from matplotlib import rc
import joblib

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from torch import nn,optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics.functional import f1
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

In [2]:
m=6000
text=[]
summary=[]
df=pd.read_csv('/kaggle/input/arabic-text-summarization-30-000/wikiHow.csv')
for i in range(df.shape[0]):
    if len(df['text'][i])< m:
        text.append(df['text'][i])
        summary.append(df['summary'][i])
        
df = pd.DataFrame({'text': text, 'summary': summary})
df['summary'] = df['summary'].replace(r'\n', '', regex = True)
df.head()

,text,summary
0,الأطعمة الصحية ليست باهظة الثمن بالضرورة، بل ف...,فضل خيارات الأطعمة الأرخص ثمنا. تباطأ في استهل...
1,استفد من حديقتك المنزلية أو أصيص الزرع الصغير ...,ازرع كل ما يمكنك من خضراوات وفاكهة. اطه بنفسك ...
2,تساعدك الخطط المسبقة في كل نواحي حياتك على وضع...,خطط مسبقا لوجباتك الرئيسية لمدة أسبوع. التزم ب...
3,نظرا لأن السبب الرئيسي لضغط العين هو أن ثقافة ...,قلل وقت التعرض للشاشات. اذهب إلى الطبيب.
4,افعل ذلك وأنت جالس لمنع نفسك من النوم. أغلق عي...,أغلق عينيك.


In [3]:
train, val = train_test_split(df, test_size=0.1, random_state=42)

train.to_csv("train.csv",index=False)
val.to_csv("val.csv",index=False)

In [5]:
class SummaryDataset(Dataset):
    def __init__(
        self,
        data,
        text_max_token_len = 2000,
        summary_max_token_len = 150
    ):
        self.tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['text']

        text_encoding = self.tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = self.tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

In [6]:
class SummaryDataModule(pl.LightningDataModule):
    def __init__(self,train_path,val_path,batch_size=12, text_max_token_len = 2000, summary_max_token_len = 150):
        super().__init__()
        self.train_path,self.val_path= train_path,val_path
        self.batch_size = batch_size
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def setup(self,stage=None):
        train = pd.read_csv(self.train_path)
        val = pd.read_csv(self.val_path)
        self.train_dataset = SummaryDataset(data=train,
                                            text_max_token_len=self.text_max_token_len,
                                            summary_max_token_len=self.summary_max_token_len)
        self.val_dataset = SummaryDataset(data=val,
                                          text_max_token_len=self.text_max_token_len,
                                          summary_max_token_len=self.summary_max_token_len)
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, shuffle=True, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, shuffle=False, num_workers=4)


In [7]:
class T5SumModel(pl.LightningModule):
    def __init__(self, lr=0.0001):
        super().__init__()
        self.lr = lr
        self.model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
    
    def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):
        output = self.model(
            input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask
        )
        return output.loss, output.logits
    
    def configure_optimizers(self):
        return optim.AdamW(self.parameters(), lr=self.lr)
    
    def training_step(self, batch, batch_size):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        decoder_attention_mask = batch['decoder_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_size):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        decoder_attention_mask = batch['decoder_attention_mask']

        loss, outputs = self(
            input_ids=input_ids,
            attention_mask=attention_mask,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels
        )

        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

In [8]:
dm = SummaryDataModule(train_path="./train.csv",
                val_path = "./val.csv",
                text_max_token_len = 1000, #largest length kaggle's GPUs can handle
                batch_size=1)

# First Time only
trained_model = T5SumModel() 


# Traing
trainer = pl.Trainer(gpus=-1, max_epochs=2, default_root_dir='.', callbacks=[EarlyStopping(monitor="val_loss")])
trainer.fit(trained_model,dm)

Downloading:   0%|          | 0.00/730 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [9]:
# Save the model
checkpoint_path = "/kaggle/working/checkpoint.ckpt"
trained_model.model.save_pretrained(checkpoint_path)


In [10]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")



def summarizeText(text, mymodel):
    text_encoding = tokenizer(
        text,
        max_length=1000,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors='pt'
    )
    generated_ids = mymodel.generate(
        input_ids=text_encoding['input_ids'],
        attention_mask=text_encoding['attention_mask'],
        max_length=150,
        num_beams=2,
        repetition_penalty=2.5,
        length_penalty=1.0,
        early_stopping=True
    )

    preds = [
            tokenizer.decode(gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for gen_id in generated_ids
    ]
    return "".join(preds)

In [11]:
test = val.sample(10)
for i in range(len(test)):
    sample_row = test.iloc[i]
    text = sample_row['text']
    print('Text : ', text)
    print('\n')
    print('Summary (Ground truth) : ',sample_row['summary'])
    print('\n')
    summary_without_tuning = summarizeText(text, model)
    print("Summary before tuning", summary_without_tuning)
    print('\n')
    summary_with_tuning = summarizeText(text, trained_model.model)
    print("Summary after tuning", summary_with_tuning)
    print('\n')
    print('-------------------------------------------------------------------------------------------------')

Text :  لو اعتقدت أنك تعرف دور صديقك، فمن الممكن أن يصيبك الإحباط وخيبة الأمل. قد يكون صديقك المقرب واحدا من أكثر الأشخاص قيمة في حياتك، لكنه لن يساعدك ويدعمك في كل جوانب حياتك. فلا تتوقع أن يتواجد دوما من أجلك وأن يسمعك ما ترغب في سماعه. ستصيبك التوقعات العالية جدا لدور صديقك في حياتك بالإحباط والشعور بالخيبة.  كن مسؤولا عن نفسك. أنت صديق نفسك المفضل، وأنت المسؤول عن رعاية ذاتك. إذا كنت كذلك، فلن تضع صديقك في موقف يجعله يسرع للتدخل وإنقاذك. وإن حدث هذا، فلن تخيب آمالك يوما. تذكر أنه ما من شخص كامل، حتى صديقك المفضل. لكل شخص منا أخطائه، التي يحتاج المساعدة لإصلاحها. لا تتصرف بحدة تجاه صديقك بسبب أخطائه، بل ساعده على إصلاحها، واطلب منه مساعدتك على إصلاح أخطائك أيضا. ومع ذلك عليك الحذر حين تخبر صديقك عن خطأ فعله، حتى لا تؤذي مشاعره، ومن الأفضل أن تركز فقط على الأخطاء الكبيرة التي تشكل تهديدا على صداقتكما. وميز الأخطاء التي يمكن تجاهلها والأخرى التي يتعين عليك مساعدة صديقك لإصلاحها. في بعض الأحيان قد تحتاج إلى ترك صديقك ليصلح خطئه دون مساعدتك -إلا إذا طلب منك ذلك-. أما التذمر المستمر، فسي